In [30]:
lines=[]
for i in range(17):
    if i < 10:
        j = '0'+str(i)
    else:
        j = str(i)
    f = open('../../alert.full.maccdc2012_000'+j+'.pcap','r')
    lines.append(f.readlines())
    print(len(lines[i]))

1144879
380932
91765
46519
489802
0
534753
589684
386575
608430
1233926
341013
1231744
199433
1029444
2416650
906679


In [31]:
g = open('../../alert.full.maccdc2012_00000.pcap','r')
len(g.readlines())

1144879

In [32]:
for j in range(17):
    print(j)
    for i in range(1,20):
        if j != 5:
            print(lines[j][-i])        

0


TCP Options (3) => NOP NOP TS: 646535267 1343654 

***A**** Seq: 0x35E70F53  Ack: 0x3C19364E  Win: 0xB5  TcpLen: 32

TCP TTL:63 TOS:0x0 ID:22287 IpLen:20 DgmLen:1500 DF

03/16-08:30:07.770000 192.168.27.253:443 -> 192.168.202.110:49653

[Classification: Potential Corporate Privacy Violation] [Priority: 1] 

[**] [1:2013659:3] ET POLICY Self Signed SSL Certificate (SomeOrganizationalUnit) [**]



[Xref => http://www.networkforensics.com/2010/05/16/network-detection-of-x86-buffer-overflow-shellcode/]

***A**** Seq: 0x2CD9A251  Ack: 0x8E7AD34A  Win: 0x3908  TcpLen: 20

TCP TTL:64 TOS:0x0 ID:6111 IpLen:20 DgmLen:1420 DF

03/16-08:30:07.760000 192.168.202.68:8080 -> 192.168.24.100:1038

[Classification: Executable Code was Detected] [Priority: 1] 

[**] [1:2012088:1] ET SHELLCODE Possible Call with No Offset TCP Shellcode [**]



TCP Options (3) => NOP NOP TS: 646535185 1343633 

***A**** Seq: 0x35DA9994  Ack: 0x7277A9A3  Win: 0xB5  TcpLen: 32

TCP TTL:63 TOS:0x0 ID:49566 IpLen:20 DgmLe

We are looking to find attributes to match to the main log. Here we will use the ip addresses and port numbers as well as the time. From the above the line containing this data all start with '03/', while it looks like no other lines start the same. As such I will use this to access this data from here. We will now look to filter the lines and then look to format each line into a table.

In [33]:
filtered = []
for j in range(17):
    if j != 5:
        for i in range(0,len(lines[j])-1):
            if lines[j][i].startswith('03/'):
                filtered.append(lines[j][i])

In [34]:
print(len(filtered))
s = 0
for j in range(17):
    s = s + len(lines[j])
print(s)
print(s/len(filtered))

1682304
11632228
6.9144625466027545


Here there are approximately 7 lines for each line that starts with '03/' which looking at the data looks like a good approximation. As such, I am reasonably confident that it has picked up all the lines it needed to.  

In [74]:
day=[]
hour=[]
minute=[]
seconds=[]
milli=[]
srcIP=[]
srcPort=[]
dstIP=[]
dstPort=[]
IPv4=[]
Port=[]
num1=[]
num2=[]
num3=[]
for i in range(0,len(filtered)):
    day.append(filtered[i][3:5])
    hour.append(filtered[i][6:8])
    minute.append(filtered[i][9:11])
    seconds.append(filtered[i][12:14])
    milli.append(filtered[i][15:21])
    if filtered[i][25] == '.':
        IPv4.append(True)
        if (filtered[i][22:].find(':') < filtered[i][22:].find('->')) and (filtered[i][22:].find(':') != -1):
            Port.append(True)
            x1 = filtered[i][22:].find(':') + 22
            srcIP.append(filtered[i][22:x1])
            x2 = filtered[i][22:].find('->') + 22
            srcPort.append(filtered[i][x1+1:x2-1])
            x3 = x2 + 3
            x4 = filtered[i][x3:].find(':') + x3
            dstIP.append(filtered[i][x3:x4])
            dstPort.append(filtered[i][x4+1:-1])
        else:
            num1.append(i)
            Port.append(False)
            x1 = filtered[i][22:].find('->') + 22
            srcIP.append(filtered[i][22:x1-1])
            dstIP.append(filtered[i][x1+3:-1])  
            srcPort.append(0)
            dstPort.append(0)
    else:
        IPv4.append(False)
        arrow = filtered[i][22:].find('->') + 22
        if filtered[i][22:arrow].count(':') == 8:
            num2.append(i)
            Port.append(True)
            x1 = filtered[i][22:arrow].rfind(':') + 22
            srcIP.append(filtered[i][22:x1])
            x2 = arrow
            srcPort.append(filtered[i][x1+1:x2-1])
            x3 = x2 + 3
            x4 = filtered[i][x3:].rfind(':') + x3
            dstIP.append(filtered[i][x3:x4])
            dstPort.append(filtered[i][x4+1:-1])
        else:
            num3.append(i)
            Port.append(False)
            x1 = arrow
            srcIP.append(filtered[i][22:x1-1])
            dstIP.append(filtered[i][x1+3:-1])
            srcPort.append(0)
            dstPort.append(0)

In [73]:
for i in range(5):
    print(filtered[num2[i]])
    print(srcIP[num2[i]])
    print(srcPort[num2[i]])
    print(dstIP[num2[i]])
    print(dstPort[num2[i]])
    print('/')
    print(filtered[num1[i]])
    print(srcIP[num1[i]])
    print(dstIP[num1[i]])
    print('/')
    print(filtered[num3[i]])
    print(srcIP[num3[i]])
    print(dstIP[num3[i]])
    print('/')

03/16-11:22:13.200000 2001:dbb:c18:202:20c:29ff:fe93:571e:60681 -> 2001:dbb:c18:1:216:47ff:fe9d:f2d7:3306

2001:dbb:c18:202:20c:29ff:fe93:571e
60681
2001:dbb:c18:1:216:47ff:fe9d:f2d7
3306
/
03/16-07:30:00.060000 192.168.27.25 -> 192.168.202.100

192.168.27.25
192.168.202.100
/
03/16-07:31:58.510000 :: -> ff02::1:ff8e:385a

::
ff02::1:ff8e:385a
/
03/16-11:22:13.280000 2001:dbb:c18:202:20c:29ff:fe93:571e:60681 -> 2001:dbb:c18:1:216:47ff:fe9d:f2d7:1433

2001:dbb:c18:202:20c:29ff:fe93:571e
60681
2001:dbb:c18:1:216:47ff:fe9d:f2d7
1433
/
03/16-07:30:00.300000 192.168.27.103 -> 192.168.202.100

192.168.27.103
192.168.202.100
/
03/16-07:34:33.350000 :: -> ff02::1:ff8e:385a

::
ff02::1:ff8e:385a
/
03/16-11:22:13.280000 2001:dbb:c18:202:20c:29ff:fe93:571e:60681 -> 2001:dbb:c18:1:216:47ff:fe9d:f2d7:161

2001:dbb:c18:202:20c:29ff:fe93:571e
60681
2001:dbb:c18:1:216:47ff:fe9d:f2d7
161
/
03/16-07:30:00.500000 192.168.202.1 -> 192.168.202.100

192.168.202.1
192.168.202.100
/
03/16-07:38:16.020000 :: -

The above makes me confident in the ability for the code to separate the relevant information regarless of IPv and if ports are included. We will now format the time. We will use that the first packet in the snort log occurs at the first time instance in conn.log (time = 1331901000.000000).

In [103]:
time=[]
scaler16 = (int(milli[0]) / 1000000) + int(seconds[0]) + int(minute[0]) * 60 + int(hour[0]) * 3600
scaler17 = (int(milli[1175157]) / 1000000) + int(seconds[1175157]) + int(minute[1175157]) * 60 + int(hour[1175157]) * 3600 #1037495
for i in range(len(filtered)):
    if day[i] == 16:
        temp = (int(milli[i]) / 1000000) + int(seconds[i]) + int(minute[i]) * 60 + int(hour[i]) * 3600 - scaler16 + 1331901000
    else:
        temp = (int(milli[i]) / 1000000) + int(seconds[i]) + int(minute[i]) * 60 + int(hour[i]) * 3600 - scaler17 +  1331997339.739999#1331987253.53
    time.append(temp) 

In [101]:
for i in range (1,5):
    print(time[-i])
    print(filtered[-i])

1332017981.32
03/17-15:59:41.320000 192.168.202.1 -> 192.168.202.80

1332017979.59
03/17-15:59:39.590000 192.168.1.254:1900 -> 239.255.255.250:1900

1332017979.59
03/17-15:59:39.590000 192.168.1.254:1900 -> 239.255.255.250:1900

1332017979.59
03/17-15:59:39.590000 192.168.1.254:1900 -> 239.255.255.250:1900



In [106]:
for i in range(len(filtered)):
    if (filtered[i].startswith('03/17-07:27')):
        print('new')
        print(filtered[i])
        print(time[i])
        print(i)

new
03/17-07:27:32.800000 192.168.202.118 -> 192.168.202.1

1331988192.4
1037489
new
03/17-07:27:32.800000 192.168.202.118 -> 192.168.202.1

1331988192.4
1037490
new
03/17-07:27:32.800000 192.168.202.118 -> 192.168.202.1

1331988192.4
1037491
new
03/17-07:27:32.800000 192.168.202.118 -> 192.168.202.1

1331988192.4
1037492
new
03/17-07:27:32.800000 192.168.202.118 -> 192.168.202.1

1331988192.4
1037493
new
03/17-07:27:32.800000 192.168.202.1 -> 192.168.202.118

1331988192.4
1037494
new
03/17-07:27:33.530000 192.168.202.87:50082 -> 192.168.21.116:1521

1331988193.13
1037495
new
03/17-07:27:33.800000 192.168.202.118 -> 192.168.202.1

1331988193.4
1037496
new
03/17-07:27:33.800000 192.168.202.118 -> 192.168.202.1

1331988193.4
1037497
new
03/17-07:27:33.800000 192.168.202.118 -> 192.168.202.1

1331988193.4
1037498
new
03/17-07:27:33.800000 192.168.202.118 -> 192.168.202.1

1331988193.4
1037499
new
03/17-07:27:33.800000 192.168.202.118 -> 192.168.202.1

1331988193.4
1037500
new
03/17-07:27:


1331988218.43
1037963
new
03/17-07:27:58.830000 192.168.202.138 -> 192.168.25.19

1331988218.43
1037964
new
03/17-07:27:58.830000 192.168.202.138 -> 192.168.25.19

1331988218.43
1037965
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.20

1331988218.44
1037966
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.20

1331988218.44
1037967
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.21

1331988218.44
1037968
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.21

1331988218.44
1037969
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.22

1331988218.44
1037970
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.22

1331988218.44
1037971
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.25

1331988218.44
1037972
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.25

1331988218.44
1037973
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.26

1331988218.44
1037974
new
03/17-07:27:58.840000 192.168.202.138 -> 192.168.25.26

1331988218.44
1

1331988218.94
1038353
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.115

1331988218.94
1038354
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.116

1331988218.94
1038355
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.116

1331988218.94
1038356
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.117

1331988218.94
1038357
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.117

1331988218.94
1038358
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.120

1331988218.94
1038359
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.120

1331988218.94
1038360
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.2

1331988218.94
1038361
new
03/17-07:27:59.340000 192.168.202.138 -> 192.168.25.3

1331988218.94
1038362
new
03/17-07:27:59.350000 192.168.202.138 -> 192.168.25.13

1331988218.95
1038363
new
03/17-07:27:59.350000 192.168.202.138 -> 192.168.25.14

1331988218.95
1038364
new
03/17-07:27:59.350000 192.168.202.138 -> 192.168.25.17

1331988218.